In [1]:
##primeiro teste para importar o csv
##não necessário pois o gdal tem biblioteca propria para leitura de arquivo
##inclusive não funcionou no gdal tentando importar o arquivo dessa forma
#import pandas as pd
#url = 'https://raw.githubusercontent.com/climatempo/challenge-accepted-gis/master/temperatura_maxmin_GIS.csv'
#header_list = ['Local', 'Longitude', 'Latitude', 'Temperatura', 'minmax']
#df = pd.read_csv(url, sep = ';', header = None, names=header_list)
#print(df)

In [2]:
##Código adaptado do Python GDAL/OGR Cookbook 1.0 documentation disponível em:##
##https://pcjericks.github.io/py-gdalogr-cookbook/vector_layers.html#read-a-csv-of-coordinates-as-an-ogrvrtlayer

import osgeo.ogr as ogr
import osgeo.osr as osr

##necessário para importar o arquivo .csv de forma apropriada para uso no gdal
##código disponível no gdal não funcionou, tive que pesquisar alguns parâmetros para que funcionasse adequadamente
import csv

# use a dictionary reader so we can access by field name
##adicionei o cabeçalho pois não não foi possível criar os atributos apenas lendo a posição das colunas do vetor
##só funcionou com nomes curtos(temperatura, por exemplo, 
##não estava importando o valor correspondente na tabela de atributos, os campos estavam nulos)

fieldnames = ['local', 'lon', 'lat', 'tmp', 'minmax'] 

##tentativa usando a importação do csv com pandas
##reader = df 

##tive problemas para o código funcionar sem realizar o download do arquivo
reader = csv.DictReader(open("C:/tmp/temperatura_maxmin_GIS.csv","rt",encoding='utf-8'), 
    fieldnames=fieldnames,
    delimiter=';',   
    quoting=csv.QUOTE_NONE)


# set up the shapefile driver
driver = ogr.GetDriverByName("ESRI Shapefile")

# create the data source

##funcionou encurtando o caminho para a pasta e deixando o diretório vazio
data_source = driver.CreateDataSource("C:/tmp/temperaturas.shp")

# create the spatial reference, WGS 84
##alterado para SIRGAS 2000 - datum oficial
srs = osr.SpatialReference()
srs.ImportFromEPSG(4674)

# create the layer
layer = data_source.CreateLayer("temperaturas", srs, ogr.wkbPoint)


In [3]:
##quebrei o código de exemplo em duas partes porque não estava funcionando e eu precisava isolar o problema
##não estava criando o arquivo .shp (linha 34 do quadro acima)

# Add the fields we're interested in
##só funcionou com os nomes curtos
field_name = ogr.FieldDefn("local", ogr.OFTString)
field_name.SetWidth(24)
layer.CreateField(field_name)
layer.CreateField(ogr.FieldDefn("lon", ogr.OFTReal))
layer.CreateField(ogr.FieldDefn("lat", ogr.OFTReal))
field_name = ogr.FieldDefn("tmp", ogr.OFTString)
field_name.SetWidth(24)
layer.CreateField(field_name)
field_name = ogr.FieldDefn("minmax", ogr.OFTString)
field_name.SetWidth(24)
layer.CreateField(field_name)
field_name = ogr.FieldDefn("cond", ogr.OFTString)
field_name.SetWidth(24)
layer.CreateField(field_name)

# Process the text file and add the attributes and features to the shapefile
## Etapa plus de classificação das temperaturas antes de adicionar todos os atributos ao shp
for row in reader:
    
    # create the feature
    
    ##não deu certo inicializar a variável nula  
    cond = 'x'
    
    if float(row['tmp'])<=20:
        cond = 'low'
    elif float(row['tmp'])>20 and float(row['tmp'])<=30:
        cond = 'medium'
    elif float(row['tmp'])>30 and float(row['tmp'])<40:
        cond = 'high'
    else:
        cond = 'extreme'
           
            
    feature = ogr.Feature(layer.GetLayerDefn())
    # Set the attributes using the values from the delimited text file
    feature.SetField("local", row['local'])
    feature.SetField("lon", row['lon'])
    feature.SetField("lat", row['lat'])
    feature.SetField("tmp", row['tmp'])
    feature.SetField("minmax", row['minmax'])
    feature.SetField("cond",cond)
    
    # create the WKT for the feature using Python string formatting
    wkt = "POINT(%f %f)" %  (float(row['lon']) , float(row['lat']))
    
    # Create the point from the Well Known Txt
    point = ogr.CreateGeometryFromWkt(wkt)

    # Set the feature geometry using the point
    feature.SetGeometry(point)
    # Create the feature in the layer (shapefile)
    layer.CreateFeature(feature)
    # Dereference the feature
    feature = None

# Save and close the data source
data_source = None

In [4]:
##Transformando shp em geojson
##Tentativa com geopandas não foi utilizada, estou com dificuldades para instalar a biblioteca
##Provavelmente conflito com a versão do gdal que foi instalada com o QGis

from osgeo import gdal


gdal.UseExceptions()

##importantando o .shp gerado na etapa anterior
srcDS = gdal.OpenEx('C:/tmp/temperaturas.shp')
ds = gdal.VectorTranslate('C:/tmp/temperaturas.geojson', srcDS=srcDS, format = 'GeoJSON', layerCreationOptions = ['RFC7946=YES', 'WRITE_BBOX=YES'])

#Dereference and close dataset, then reopen.
del ds
ds = gdal.OpenEx('output.geojson')

##conferindo se foram importados todos os pontos
#print(srcDS.GetLayer(0).GetFeatureCount())
#print(ds.GetLayer(0).GetFeatureCount())